In [1]:
import pickle
import gzip
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
import os
import numpy as np
np.random.seed(666)

from tensorflow import set_random_seed
set_random_seed(125)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

from keras.datasets import mnist
import keras

from keras.layers import Conv2D,MaxPooling2D,Flatten


Using TensorFlow backend.


In [149]:
def accuracy(predicted,target):
    correct = 0
    left = np.array(np.argmax(predicted,axis=0)).flatten()
    right = np.argmax(target,axis=1)
    confusion_mat = np.zeros((len(np.unique(right)),len(np.unique(left))))
    for i in range(len(target)):
        if(left[i] == right[i]):
            correct+=1
        confusion_mat[left[i]][right[i]] =confusion_mat[left[i]][right[i]] +1
    return correct/len(target),pd.DataFrame(np.matrix(confusion_mat,dtype="int32"))

def more_metrics(conf_mat):
    true_positives = 0
    precision = []
    recall = []
    for i in range(len(conf_mat)):
        true_positives += conf_mat.iloc[i,i]
    conf_mat = np.matrix(conf_mat)
    tp_fp = np.array(np.sum(conf_mat,axis=1)).ravel()
    relevant_elements = np.array(np.sum(conf_mat,axis=0)).ravel()
    for i in range(len(conf_mat)):
        precision.append(conf_mat[i,i]/tp_fp[i])
        recall.append(conf_mat[i,i]/relevant_elements[i])
    return true_positives,precision,recall

In [136]:
filename = '../mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()
train_data = np.append(training_data[0],validation_data[0],axis=0)
train_target = np.append(training_data[1],validation_data[1],axis=0)
train_target = keras.utils.to_categorical(train_target, 10)
test_target = keras.utils.to_categorical(test_data[1], 10)
test_data = test_data[0]

In [135]:
train_data = train_data.reshape(train_data.shape, image_vector_size)
test_data = test_data.reshape(test_data.shape[0], image_vector_size)


60000

In [117]:
def baseline():
    model = Sequential()
    model.add(Dense(units=32, activation='sigmoid', input_shape=(image_vector_size,)))
    model.add(Dense(units=num_classes, activation='softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

def highHidden():
    drop_out = 0.1
    model = Sequential()
    model.add(Dense(units=32,activation='sigmoid',input_shape=(image_vector_size,)))
    model.add(Activation('tanh'))
    model.add(Dense(128, input_dim=image_vector_size))
    model.add(Activation('relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(128, input_dim=image_vector_size))
    model.add(Dense(units=num_classes, activation='softmax'))

    
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])    
    model.summary()
    return model

def lowHidden():
    model = Sequential()
    model.add(Dense(units=1024,activation='tanh',input_shape=(image_vector_size,)))
    model.add(Dense(2048,activation='relu',input_dim=image_vector_size))
    model.add(Dense(units=num_classes, activation='softmax'))

    
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])    
    model.summary()
    return model

def conv():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),activation='tanh',input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model

In [10]:
img_x, img_y = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [137]:
model = lowHidden()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 1024)              803840    
_________________________________________________________________
dense_51 (Dense)             (None, 2048)              2099200   
_________________________________________________________________
dense_52 (Dense)             (None, 10)                20490     
Total params: 2,923,530
Trainable params: 2,923,530
Non-trainable params: 0
_________________________________________________________________


In [138]:
history = model.fit(x_train, y_train, batch_size=128, epochs=150,verbose=True,validation_split=.1)

ValueError: Error when checking input: expected dense_50_input to have 2 dimensions, but got array with shape (60000, 28, 28, 1)

In [139]:
earlystopping_cb = EarlyStopping(monitor='acc', verbose=1, 
                                 patience=150, mode='min')

history = model.fit(train_data, train_target, 
                    batch_size=128, 
                    epochs=150,
                    verbose=True,
                    validation_split=.1,
                   callbacks = [earlystopping_cb])

Train on 54000 samples, validate on 6000 samples
Epoch 1/150
54000/54000 [==============================] - 11s 204us/step - loss: 0.2092 - acc: 0.9358 - val_loss: 0.0975 - val_acc: 0.9712
Epoch 2/150
54000/54000 [==============================] - 10s 184us/step - loss: 0.0970 - acc: 0.9693 - val_loss: 0.1031 - val_acc: 0.9720
Epoch 3/150
54000/54000 [==============================] - 10s 184us/step - loss: 0.0744 - acc: 0.9762 - val_loss: 0.1067 - val_acc: 0.9720
Epoch 4/150
54000/54000 [==============================] - 10s 184us/step - loss: 0.0641 - acc: 0.9803 - val_loss: 0.0894 - val_acc: 0.9750
Epoch 5/150
54000/54000 [==============================] - 10s 184us/step - loss: 0.0533 - acc: 0.9824 - val_loss: 0.0968 - val_acc: 0.9753
Epoch 6/150
54000/54000 [==============================] - 10s 185us/step - loss: 0.0444 - acc: 0.9865 - val_loss: 0.0941 - val_acc: 0.9775
Epoch 7/150
54000/54000 [==============================] - 10s 184us/step - loss: 0.0456 - acc: 0.9857 - val_lo

KeyboardInterrupt: 

In [140]:
loss,accuracy = model.evaluate(x_test, y_test, verbose=True)

ValueError: Error when checking input: expected dense_50_input to have 2 dimensions, but got array with shape (10000, 28, 28, 1)

In [157]:
def accuracy(predicted,target):
    correct = 0
    left = np.array(np.argmax(predicted,axis=0)).flatten()
    right = np.argmax(target,axis=1)
    confusion_mat = np.zeros((len(np.unique(right)),len(np.unique(left))))
    for i in range(len(target)):
        if(left[i] == right[i]):
            correct+=1
        confusion_mat[left[i]][right[i]] =confusion_mat[left[i]][right[i]] +1
    return correct/len(target),pd.DataFrame(np.matrix(confusion_mat,dtype="int32"))

ValueError: sequence too large; cannot be greater than 32

In [177]:
correct = 0
for i in range(len(test_target)):
        if(left[i] == right[i]):
            correct+=1
        confusion_mat[left[i]][right[i]] =confusion_mat[left[i]][right[i]] +1

In [181]:
correct/len(test_target)

0.1007

In [143]:
USPSMat  = []
USPSTar  = []
curPath  = '../USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

In [91]:
USPSMat  = []
USPSTar  = []
curPath  = '../USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            img = np.asarray(img)
            #savedImg = img
            #imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(img)
            USPSTar.append(j)

In [92]:
USPSMat = np.asarray(USPSMat)
USPSMat = USPSMat.reshape(USPSMat.shape[0], 28, 28, 1)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)

In [144]:
USPSTar = keras.utils.to_categorical(USPSTar, num_classes)

In [51]:
train_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [145]:
loss,accuracy = model.evaluate([USPSMat], USPSTar, verbose=True)

19999/19999 [==============================] - 2s 88us/step


In [146]:
loss

6.493300959708554

In [147]:
accuracy

0.49107455372023545